In [1]:
# Imports
from biocrnpyler import *
from genelet import *
import pylab as plt
import numpy as np
from bokeh.layouts import row

import warnings
import bokeh.io
import bokeh.plotting
from bioscrape.simulator import py_simulate_model
from bioscrape.sbmlutil import import_sbml
from bioscrape.types import Model

## CRNs
### NAND Gate CRN

In [3]:
# Creating CRN for NAND gate

S1_off = Species("Sw1")
S2_off = Species("Sw2")
S3_off = Species("Sw3")

S1 = Genelet("Sw1", transcript = "I31", activator = "A1", inhibitor = "I1" )
S2 = Genelet("Sw2", transcript = "I32", activator = "A2", inhibitor = "I2" )
S3 = Genelet("Sw3", transcript = "I4", activator = "A31", inhibitor = "I31", activator2 = "A32", inhibitor2 = "I32" )

M_NAND = Mixture(name = "Switch_test", components = [S1,S2,S3], parameter_file = "default_parameters.txt")

repr(M_NAND)
CRN_NAND = M_NAND.compile_crn()

#CRN_NAND.write_sbml_file('Parametric Analysis/NAND_CRN.xml')
#print(CRN_NAND)

### AND Gate CRN

In [4]:
S4_off = Species("Sw4")

S4 = Genelet(S4_off, transcript = "P", activator = "A4", inhibitor = "I4" )

M_AND = Mixture(name = "AND_test", components = [S1,S2,S3,S4], parameter_file = "default_parameters.txt")

repr(M_AND)
CRN_AND = M_AND.compile_crn()
#print(CRN_AND.pretty_print())

### NOR Gate CRN

In [5]:
# Creating CRN for NOR gate

S1_off = Species("Sw1")
S2_off = Species("Sw2")
S3_off = Species("Sw3")

S1 = Genelet(S1_off, transcript = "I3", activator = "A1", inhibitor = "I1" )
S2 = Genelet(S2_off, transcript = "I3", activator = "A2", inhibitor = "I2" )
S3 = Genelet(S3_off, transcript = "I4", activator = "A3", inhibitor = "I3" )

M_NOR = Mixture(name = "Switch_test", components = [S1,S2,S3], parameter_file = "default_parameters.txt")

repr(M_NOR)
CRN_NOR = M_NOR.compile_crn()
#CRN_NOR.write_sbml_file('Parametric Analysis/NOR_CRN.xml')
#print(CRN_NOR)

### OR Gate CRN

In [6]:
S4_off = Species("Sw4")

S4 = Genelet(S4_off, transcript = "P", activator = "A4", inhibitor = "I4" )

M_OR = Mixture(name = "AND_test", components = [S1,S2,S3,S4], parameter_file = "default_parameters.txt")

repr(M_OR)
CRN_OR = M_OR.compile_crn()
#print(CRN_OR.pretty_print())

## Simulations
### NAND Gate Simulation
#### Inhibitor added at t = 10 min

In [7]:
M_NAND = import_sbml("Parametric Analysis/NAND_CRN.xml")

#Create rule 1:
#NOTE Rules can also be passed into the Model constructor as a list of tuples [("rule_type", {"equation":"eq string"})]
M_NAND.create_rule("assignment", {"equation":"rna_I1 = _I0*Heaviside(t-_T_I0)"}) #"_" must be placed before param names, but not species.

M_NAND.create_parameter("I0", 500) #Inducer concentration
M_NAND.create_parameter("T_I0", 600) #Initial time inducer is added

#Create rule 1:
#NOTE Rules can also be passed into the Model constructor as a list of tuples [("rule_type", {"equation":"eq string"})]
M_NAND.create_rule("assignment", {"equation":"rna_I1 = _I0*Heaviside(t-_T_I0)"}) #"_" must be placed before param names, but not species.

M_NAND.set_species({"Sw1_OFF": 2000, "dna_A1": 2000, "rna_I1": 0, "Sw2_OFF": 2000, "dna_A2": 2000, "rna_I2": 0, "Sw3_OFF":2000, "dna_A31": 2000, "dna_A32": 2000, "protein_RNAseH":70,
      "protein_RNAP":500})


timepoints = np.linspace(0, 3000, 1000)
R = py_simulate_model(timepoints, Model = M_NAND) #Returns a Pandas DataFrame
timepoints = timepoints/60

bokeh.io.output_notebook()
p = bokeh.plotting.figure(plot_width=400, plot_height=400, x_axis_label='Time (min)',
    y_axis_label='Concentration (uM)')
p.circle(timepoints, R["Sw1_OFF"], legend_label = "OFF switch 1", color = "orange")
p.circle(timepoints, R["complex_Sw1_ON"], legend_label = "ON switch 1", color = "red")

s = bokeh.plotting.figure(plot_width=400, plot_height=400, x_axis_label='Time (min)')
s.circle(timepoints, R["Sw2_OFF"], legend_label = "OFF switch 1", color = "orange")
s.circle(timepoints, R["complex_Sw2_ON"], legend_label = "ON switch 2" , color = "red")

q = bokeh.plotting.figure(plot_width=400, plot_height=400, x_axis_label='Time (min)')
q.circle(timepoints, R["Sw3_OFF"], legend_label = "OFF switch 3", color = "orange")
q.circle(timepoints, R["complex_Sw3_ON_1"]+R["complex_Sw3_ON_2"], legend_label = "ON switch 3" , color = "red")

# NOR gate activators and inhibitors of importance

r = bokeh.plotting.figure(plot_width=400, plot_height=400, x_axis_label='Time (min)')
r.circle(timepoints, R["dna_A31"]+R["dna_A32"], legend_label = "Activator 3", color = "orange")
r.circle(timepoints, R["rna_I31"], legend_label = "Inhibitor 31", color = "red")
r.circle(timepoints, R["rna_I32"], legend_label = "Inhibitor 32", color = "purple")

r.circle(timepoints, R["complex_Sw3_AI"]+R["complex_Sw3_AI_2"], legend_label = "Complex 3", color = "blue")
r.legend.click_policy="hide"



bokeh.io.show(row(p, s, q, r))
warnings.filterwarnings("ignore")




/home/nazmus/anaconda3/lib/python3.7/site-packages/bioscrape/sbmlutil.py:209: UserWarning: Compartments, UnitDefintions, Events, and some other SBML model components are not recognized by bioscrape. Refer to the bioscrape wiki for more information.
  warnings.warn('Compartments, UnitDefintions, Events, and some other SBML model components are not recognized by bioscrape. ' +
/home/nazmus/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Uninitialized Model Passed into ModelCSimInterface. Model.py_initialize() called automatically.
/home/nazmus/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
odeint failed with mxstep=500...

Loading BokehJS ...

#### No addition of species after t = 0

In [8]:
# Bioscrape simulation of above CRN

io = {"Sw1_OFF": 2000, "dna_A1": 2000, "rna_I1": 0, "Sw2_OFF": 2000, "dna_A2": 2000, "rna_I2": 0, "Sw3_OFF":2000, "dna_A31": 2000, "dna_A32": 2000, "protein_RNAseH":20,
      "protein_RNAP":150}
timepoints = np.linspace(0, 5000, 1000)
R = CRN_NAND.simulate_with_bioscrape(timepoints, initial_condition_dict = io)
timepoints = timepoints/60

bokeh.io.output_notebook()
p = bokeh.plotting.figure(plot_width=300, plot_height=300, x_axis_label='Time (min)',
    y_axis_label='Concentration (uM)')
p.circle(timepoints, R["Sw1_OFF"], legend_label = "OFF switch 1", color = "orange")
p.circle(timepoints, R["complex_Sw1_ON"], legend_label = "ON switch 1", color = "red")
p.legend.click_policy="hide"


s = bokeh.plotting.figure(plot_width=300, plot_height=300, x_axis_label='Time (min)')
s.circle(timepoints, R["Sw2_OFF"], legend_label = "OFF switch 2", color = "orange")
s.circle(timepoints, R["complex_Sw2_ON"], legend_label = "ON switch 2" , color = "red")
s.legend.click_policy="hide"


# Total amount of activated and deactivated forms of Switch 3
q = bokeh.plotting.figure(plot_width=300, plot_height=300, x_axis_label='Time (min)')
q.circle(timepoints, R["Sw3_OFF"], legend_label = "OFF switch 3", color = "orange")
q.circle(timepoints, R["complex_Sw3_ON_1"]+R["complex_Sw3_ON_2"], legend_label = "ON switch 3" , color = "red")
q.legend.click_policy="hide"



r = bokeh.plotting.figure(plot_width=300, plot_height=300, x_axis_label='Time (min)')
#r.circle(timepoints, R["rna_I4"], legend_label = "Inhibitor 4", color = "green")
r.circle(timepoints, R["rna_I31"], legend_label = "Inhibitor 31" , color = "red")
#r.circle(timepoints, R["rna_I32"], legend_label = "Inhibitor 32" , color = "blue")
r.circle(timepoints, R["complex_Sw3_AI"], legend_label = "Complex 31", color = "blue")
r.circle(timepoints, R["dna_A31"], legend_label = "Complex 31", color = "orange")

r.legend.click_policy="hide"

t = bokeh.plotting.figure(plot_width=300, plot_height=300, x_axis_label='Time (min)')
t.circle(timepoints, R["Sw3_OFF"], legend_label = "OFF switch 3", color = "orange")
t.circle(timepoints, R["complex_Sw3_ON_1"], legend_label = "ON switch 3 - 1" , color = "red")
t.circle(timepoints, R["complex_Sw3_ON_2"], legend_label = "ON switch 3 - 2" , color = "blue")
t.legend.click_policy="hide"



bokeh.io.show(row(p, s, q, r, t))
warnings.filterwarnings("ignore")

odeint failed with mxstep=500...

Loading BokehJS ...

### NOR Gate Simulation
#### Inhibitor added at t = 10 min

In [14]:
M_NOR = import_sbml("Parametric Analysis/NOR_CRN.xml")

M_NOR.create_parameter("I0", 500) #Inducer concentration
M_NOR.create_parameter("T_I0", 600) #Initial time inducer is added

#Create rule 1:
#NOTE Rules can also be passed into the Model constructor as a list of tuples [("rule_type", {"equation":"eq string"})]
M_NOR.create_rule("assignment", {"equation":"rna_I1 = _I0*Heaviside(t-_T_I0)"}) #"_" must be placed before param names, but not species.

M_NOR.set_species({"Sw1_OFF": 2000, "dna_A1": 2000, "rna_I1": 0, "Sw2_OFF": 2000, "dna_A2": 0, "rna_I2": 0, "Sw3_OFF":2000, "dna_A3": 2000,
      "rna_I3": 0, "protein_RNAseH":90, "protein_RNAP":500})


timepoints = np.linspace(0, 3000, 1000)
R = py_simulate_model(timepoints, Model = M_NOR) #Returns a Pandas DataFrame
timepoints = timepoints/60

bokeh.io.output_notebook()
p = bokeh.plotting.figure(plot_width=400, plot_height=400, x_axis_label='Time (min)',
    y_axis_label='Concentration (uM)')
p.circle(timepoints, R["Sw1_OFF"], legend_label = "OFF switch 1", color = "orange")
p.circle(timepoints, R["complex_Sw1_ON"], legend_label = "ON switch 1", color = "red")

s = bokeh.plotting.figure(plot_width=400, plot_height=400, x_axis_label='Time (min)')
s.circle(timepoints, R["Sw2_OFF"], legend_label = "OFF switch 1", color = "orange")
s.circle(timepoints, R["complex_Sw2_ON"], legend_label = "ON switch 2" , color = "red")

q = bokeh.plotting.figure(plot_width=400, plot_height=400, x_axis_label='Time (min)')
q.circle(timepoints, R["Sw3_OFF"], legend_label = "OFF switch 3", color = "orange")
q.circle(timepoints, R["complex_Sw3_ON"], legend_label = "ON switch 3" , color = "red")

# NOR gate activators and inhibitors of importance

r = bokeh.plotting.figure(plot_width=400, plot_height=400, x_axis_label='Time (min)')
r.circle(timepoints, R["dna_A3"], legend_label = "Activator 3", color = "orange")
r.circle(timepoints, R["rna_I3"], legend_label = "Inhibitor 3", color = "red")
r.circle(timepoints, R["complex_Sw3_AI"], legend_label = "Complex 3", color = "blue")
r.legend.click_policy="hide"



bokeh.io.show(row(p, s, q, r))
warnings.filterwarnings("ignore")



odeint failed with mxstep=500...

Loading BokehJS ...

#### No addition of species after t = 0

In [15]:
# Bioscrape simulation of NOR gate CRN

io = {"Sw1_OFF": 2000, "dna_A1": 2000, "rna_I1": 0, "Sw2_OFF": 2000, "dna_A2": 0, "rna_I2": 0, "Sw3_OFF":2000, "dna_A3": 2000,
      "rna_I3": 0, "protein_RNAseH":10, "protein_RNAP":500}

timepoints = np.linspace(0, 1000, 1000)
R = CRN_NOR.simulate_with_bioscrape(timepoints, initial_condition_dict = io)
timepoints = timepoints/60


bokeh.io.output_notebook()
p = bokeh.plotting.figure(plot_width=400, plot_height=400, x_axis_label='Time (min)',
    y_axis_label='Concentration (uM)')
p.circle(timepoints, R["Sw1_OFF"], legend_label = "OFF switch 1", color = "orange")
p.circle(timepoints, R["complex_Sw1_ON"], legend_label = "ON switch 1", color = "red")

s = bokeh.plotting.figure(plot_width=400, plot_height=400, x_axis_label='Time (min)')
s.circle(timepoints, R["Sw2_OFF"], legend_label = "OFF switch 1", color = "orange")
s.circle(timepoints, R["complex_Sw2_ON"], legend_label = "ON switch 2" , color = "red")

q = bokeh.plotting.figure(plot_width=400, plot_height=400, x_axis_label='Time (min)')
q.circle(timepoints, R["Sw3_OFF"], legend_label = "OFF switch 3", color = "orange")
q.circle(timepoints, R["complex_Sw3_ON"], legend_label = "ON switch 3" , color = "red")

# NOR gate activators and inhibitors of importance

r = bokeh.plotting.figure(plot_width=400, plot_height=400, x_axis_label='Time (min)',
    y_axis_label='Concentration (uM)')
r.circle(timepoints, R["dna_A3"], legend_label = "Activator 3", color = "orange")
r.circle(timepoints, R["rna_I3"], legend_label = "Inhibitor 3", color = "red")
r.circle(timepoints, R["complex_Sw3_AI"], legend_label = "Complex 3", color = "blue")
r.legend.click_policy="hide"

#s = bokeh.plotting.figure(plot_width=300, plot_height=300, x_axis_label='Time (min)')
#s.circle(timepoints, R["rna_I4"], legend_label = "Inhibitor 4", color = "red")
#s.legend.click_policy="hide"

bokeh.io.show(row(p, s, q, r))
warnings.filterwarnings("ignore")

odeint failed with mxstep=500...

Loading BokehJS ...

### AND Gate Simulation

In [14]:
# Bioscrape simulation of above CRN

io = {"Sw1_OFF": 2000, "dna_A1": 2000, "rna_I1": 0, "Sw2_OFF": 2000, "dna_A2": 2000, "rna_I2": 0, "Sw3_OFF":2000, "dna_A31": 2000, "dna_A32": 2000, "Sw4_OFF":2000,
      "dna_A4": 2000, "protein_RNAseH":70, "protein_RNAP":500}
timepoints = np.linspace(0, 4000, 1000)
R = CRN_AND.simulate_with_bioscrape(timepoints, initial_condition_dict = io)
timepoints = timepoints/60

bokeh.io.output_notebook()
p = bokeh.plotting.figure(plot_width=400, plot_height=400, x_axis_label='Time (min)',
    y_axis_label='Concentration (uM)')
p.circle(timepoints, R["Sw1_OFF"], legend_label = "OFF switch 1", color = "orange")
p.circle(timepoints, R["complex_Sw1_ON"], legend_label = "ON switch 1", color = "red")
p.legend.click_policy="hide"

s = bokeh.plotting.figure(plot_width=400, plot_height=400, x_axis_label='Time (min)')
s.circle(timepoints, R["Sw2_OFF"], legend_label = "OFF switch 2", color = "orange")
s.circle(timepoints, R["complex_Sw2_ON"], legend_label = "ON switch 2" , color = "red")
s.legend.click_policy="hide"

# Total amount of activated and deactivated forms of Switch 3
q = bokeh.plotting.figure(plot_width=400, plot_height=400, x_axis_label='Time (min)')
q.circle(timepoints, R["Sw3_OFF"], legend_label = "OFF switch 3", color = "orange")
q.circle(timepoints, R["complex_Sw3_ON_1"]+R["complex_Sw3_ON_2"], legend_label = "ON switch 3" , color = "red")
q.legend.click_policy="hide"

r = bokeh.plotting.figure(plot_width=400, plot_height=400, x_axis_label='Time (min)')
r.circle(timepoints, R["Sw4_OFF"], legend_label = "OFF switch 4", color = "orange")
r.circle(timepoints, R["complex_Sw4_ON"], legend_label = "ON switch 4" , color = "red")
r.legend.click_policy="hide"

t = bokeh.plotting.figure(plot_width=400, plot_height=400)
t.circle(timepoints, R["rna_I4"], legend_label = "Inhibitor 4", color = "green")
t.circle(timepoints, R["rna_I31"], legend_label = "Inhibitor 31" , color = "red")
t.circle(timepoints, R["dna_A31"], legend_label = "Inhibitor 32" , color = "orange")
t.circle(timepoints, R["complex_Sw3_AI"], legend_label = "Complex 31", color = "blue")
t.legend.click_policy="hide"

bokeh.io.show(row(p, s, q, r))
warnings.filterwarnings("ignore")

odeint failed with mxstep=500...

Loading BokehJS ...

In [60]:
#q = bokeh.plotting.figure(plot_width=300, plot_height=300)
#q.circle(timepoints, R["Sw3_OFF"], legend_label = "OFF switch 3", color = "orange")
#q.circle(timepoints, R["complex_Sw3_ON_1"], legend_label = "ON switch 3 - 1" , color = "red")
#q.circle(timepoints, R["complex_Sw3_ON_2"], legend_label = "ON switch 3 - 2" , color = "blue")
#q.legend.click_policy="hide"